In [1]:
import numpy as np
import cv2
import sys
import os

# googlenet 사용

In [7]:
filename = './googlenet_old/fig/pelican.jpg'
img = cv2.imread(filename)

if img is None:
    sys.exit('image read failed')

### dnn Model
model = './googlenet_old/bvlc_googlenet.caffemodel' # learning된 파라미터
config = 'googlenet_old/deploy.prototxt' # 네트워크 구조

net = cv2.dnn.readNet(model, config) # 모델이 config를 받아들임

if net.empty():
    sys.exit('network read failed')

### 클래스 이름 불러오기

classNames = []
with open('./googlenet_old/classification_classes_ILSVRC2012.txt', 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
# print(classNames)

blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123), swapRB = False) # 우리의 데이터포맷을 학습된 데이터 포맷으로 맞춰줌
net.setInput(blob)
prob = net.forward()
################################################

#print(prob.shape)
out = prob.flatten() # 차원축소
#print(out.shape) 

classid = np.argmax(out)
#print(classid)
confidence = out[classid]
print(confidence)
category = classNames[classid]
print(category)

text = f'{category}({confidence*100:4.2f})%'  # 새 형식 당황ㄴ

cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)



cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()

0.87152827
pelican


# 이미지슬라이드 + googlenet

In [3]:
img_list = os.listdir('./googlenet_old/fig')

img_files = []
for i in img_list:
    img_file = './googlenet_old/fig/'+i
    img_files.append(img_file)

cv2.namedWindow('image')

### dnn Model
model = './googlenet_old/bvlc_googlenet.caffemodel' # learning된 파라미터
config = './googlenet_old/deploy.prototxt' # 네트워크 구조

net = cv2.dnn.readNet(model, config) # 모델이 config를 받아들임

if net.empty():
    sys.exit('network read failed')
### 클래스 이름 불러오기

classNames = []
with open('./googlenet_old/classification_classes_ILSVRC2012.txt', 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')


idx = 0
while True:
    img = cv2.imread(img_files[idx])
    blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123), swapRB = False) # 우리의 데이터포맷을 학습된 데이터 포맷으로 맞춰줌
    net.setInput(blob)
    prob = net.forward()
    out = prob.flatten() # 차원축소
    #print(out.shape) 

    classid = np.argmax(out)
    confidence = out[classid]
    category = classNames[classid]

    text = f'{category}({confidence*100:4.2f})%'  # 새 형식 당황ㄴ

    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)


    cv2.imshow('image', img)
    
    if cv2.waitKey(3000) == ord('q'):
        cv2.destroyAllWindows()
        break
        
    idx += 1
    if idx>=len(img_files):
        idx = 0

# opencv_face_detector

In [6]:
img = cv2.imread('./opencv_face_detector_old/fig/45.jpg')

## Tensorflow
model = './opencv_face_detector_old/opencv_face_detector_uint8.pb'
config = './opencv_face_detector_old/opencv_face_detector.pbtxt'

face_net = cv2.dnn.readNet(model, config)

if face_net.empty():
    print('net load failed')
    sys.exit()

blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123), swapRB = False)

face_net.setInput(blob)
out = face_net.forward()
#print(out.shape)

detect = out[0, 0, :, :]
#print(detect.shape)

h, w = img.shape[:2]

for i in range(detect.shape[0]):
    confidence = detect[i, 2] # 확률 추출
    
    if confidence > 0.5:
        x1 = int(detect[i, 3]*w)
        y1 = int(detect[i, 4]*h)
        x2 = int(detect[i, 5]*w)
        y2 = int(detect[i, 6]*h)

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        text = f'Face : {confidence*100:4.2f}%'
        cv2.putText(img, text, (x1, y1-2), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 1, cv2.LINE_AA)


cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()

# 웹캠으로 face detect

In [5]:
### tensor Model
model = './opencv_face_detector_old/opencv_face_detector_uint8.pb'
config = './opencv_face_detector_old/opencv_face_detector.pbtxt'

face_net = cv2.dnn.readNet(model, config) # 네트워크 연결
if face_net.empty():
    print('net load failed')
    sys.exit()

cap = cv2.VideoCapture(0) # 캡처 온
if not cap.isOpened():
    print('Video open failed')
    cap.release()
    sys.exit()

while True:
    ret, img = cap.read()
    if not ret:
        print('Video frame failed')
        break

    blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123), swapRB = False) # 우리의 데이터포맷을 학습된 데이터 포맷으로 맞춰줌
    face_net.setInput(blob)
    out = face_net.forward()
    #print(out.shape)

    detect = out[0, 0, :, :]
    #print(detect.shape)

    h, w = img.shape[:2]

    for i in range(detect.shape[0]):
        confidence = detect[i, 2] # 확률 추출
        
        if confidence > 0.5:
            x1 = int(detect[i, 3]*w)
            y1 = int(detect[i, 4]*h)
            x2 = int(detect[i, 5]*w)
            y2 = int(detect[i, 6]*h)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
            text = f'Face : {confidence*100:4.2f}%'
            cv2.putText(img, text, (x1, y1-2), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 1, cv2.LINE_AA)


    cv2.imshow('image', img)
    
    if cv2.waitKey(20) == ord('q'):
        cv2.destroyAllWindows()
        img.release()
        break


AttributeError: 'numpy.ndarray' object has no attribute 'release'